In [1]:
import csv
import math
import numpy as np
import pandas as pd
import random
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import mean_squared_error
import warnings
#suppress warnings
warnings.filterwarnings('ignore')
# Set up the constants
mu_0 = 4 * math.pi * 1e-7  # Permeability of free space

# Set up the parameters for the data
I_values = np.linspace(10,50,5)  # Values of I to use


In [2]:
x=np.linspace(-40,40,5)    # x-position of the testing point
y=np.linspace(-40,40,5)    # y-position of the testing point
H=np.linspace(10,50,5)     # Height of the wire

# coeffcients of the line given by the equation ax+by+c=0
a=[-4,-3,-2,2,3,4]   
b=[-4,-3,-2,2,3,4]   
c=[-4,-3,-2,2,3,4]   


In [3]:

# Create the header row for the CSV file
header_row = ['I', 'H', 'X', 'Y', 'a', 'b', 'c', 'B']

# Create a list to store the rows of data
data_rows = []

# Loop over the I and R values and calculate the corresponding B values
for I in I_values:
  for n in H:
    for m in x:
      for j in y:
        for q in a:
          for w in b:
            for e in c:
              R=np.abs(q*m+w*j+e)/np.sqrt(q**2+w**2)
              B = mu_0 * I / (2 * math.pi * np.sqrt(R**2+n**2))
              data_rows.append([I, n, m, j, q, w, e, B])

# Write the data to a CSV file
with open('biot_savart_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header_row)
    writer.writerows(data_rows)

data=pd.read_csv('biot_savart_data.csv', sep=',')
#data=preprocessing.normalize(data)
data=(data-data.mean())/data.std()
data

,I,H,X,Y,a,b,c,B
0,-1.414208,-1.414208,-1.414208,-1.414208,-1.28653,-1.28653,-1.286530,-1.009141
1,-1.414208,-1.414208,-1.414208,-1.414208,-1.28653,-1.28653,-0.964898,-1.009892
2,-1.414208,-1.414208,-1.414208,-1.414208,-1.28653,-1.28653,-0.643265,-1.010638
3,-1.414208,-1.414208,-1.414208,-1.414208,-1.28653,-1.28653,0.643265,-1.013579
4,-1.414208,-1.414208,-1.414208,-1.414208,-1.28653,-1.28653,0.964898,-1.014303
...,...,...,...,...,...,...,...,...
134995,1.414208,1.414208,1.414208,1.414208,1.28653,1.28653,-0.964898,-0.326841
134996,1.414208,1.414208,1.414208,1.414208,1.28653,1.28653,-0.643265,-0.328469
134997,1.414208,1.414208,1.414208,1.414208,1.28653,1.28653,0.643265,-0.334947
134998,1.414208,1.414208,1.414208,1.414208,1.28653,1.28653,0.964898,-0.336557


In [4]:
I_noise = np.random.normal(np.mean(np.array(data['I'])),1,40)
X_noise = np.random.normal(np.mean(np.array(data['X'])),1,40)
H_noise = np.random.normal(np.mean(np.array(data['H'])),1,40)
Y_noise = np.random.normal(np.mean(np.array(data['Y'])),1,40)
a_noise = np.random.normal(np.mean(np.array(data['a'])),1,40)
b_noise = np.random.normal(np.mean(np.array(data['b'])),1,40)
c_noise = np.random.normal(np.mean(np.array(data['c'])),1,40)
B_noise = np.random.normal(np.mean(np.array(data['B'])),1,40)
noise=pd.DataFrame({'I': I_noise, 'H': H_noise, 'X': X_noise, 'Y': Y_noise, 'a':a_noise, 'b':b_noise, 'c':c_noise, 'B': B_noise})
data=data.append(noise, ignore_index=True)
data=np.array(data)
print(len(data))

135040


In [5]:
X=[[row[0], row[1], row[2], row[3], row[4], row[5], row[6]] for row in data]
y=[row[7] for row in data]
# Split the data into training and testing sets with a 70/30 split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Move data to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

In [6]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, in_channels=7, num_classes=1):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = nn.Sequential(
            BasicBlock(64, 64),
            BasicBlock(64, 64),
        )
        self.layer2 = nn.Sequential(
            BasicBlock(64, 128, stride=2),
            BasicBlock(128, 128),
        )
        self.layer3 = nn.Sequential(
            BasicBlock(128, 256, stride=2),
            BasicBlock(256, 256),
        )
        self.layer4 = nn.Sequential(
            BasicBlock(256, 512, stride=2),
            BasicBlock(512, 512),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = x.view(-1, 7, 1, 1) # reshape input
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x



In [7]:
# Define the Resnet model
resnet_model = ResNet(in_channels=7, num_classes=1).to(device)

# Define the loss function
criterion = nn.L1Loss()
optimizer = torch.optim.SGD(resnet_model.parameters(), lr=0.001, weight_decay=0.001)

# Train the model
num_epochs = 10
batch_size = 128

for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(0, len(x_train), batch_size):
        # Get the inputs and labels for the current batch
        inputs = x_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item() * batch_size
    epoch_loss = running_loss / len(x_train)
    print('Epoch {} loss: {:.4f}'.format(epoch+1, epoch_loss))


Epoch 1 loss: 0.6845
Epoch 2 loss: 0.6663
Epoch 3 loss: 0.6621
Epoch 4 loss: 0.6603
Epoch 5 loss: 0.6592
Epoch 6 loss: 0.6585
Epoch 7 loss: 0.6580
Epoch 8 loss: 0.6576
Epoch 9 loss: 0.6573
Epoch 10 loss: 0.6570
